#

In [1]:
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
from PIL import Image
from deepface import DeepFace
from skeleton.inference import CenterFace
import cv2
import glob

__DATA_FOLDER = 'data/BUAA/BUAAVISNIR/'
__TARGET_SIZE = (224, 224)
__TARGET_FOLDER = 'data/buaa-w_and_wo_stripes-train_test/'

centerface = CenterFace()


def fill_black(image):
    height = image.shape[0]
    width = image.shape[1]
    max_dim = max([width, height])
    
    black_image = np.zeros((max_dim, max_dim, image.shape[-1]), dtype=np.uint8)
    
    width_padding = max_dim - width
    height_padding = max_dim - height
    y1 = int(height_padding/2)
    y2 = int(max_dim - height_padding/2)
    x1 = int(width_padding/2)
    x2 = int(max_dim - width_padding/2)
    
    black_image[y1:y2,x1:x2,:] = image
    
    return black_image
    
def operation_X(image_pth_A, image_pth_B):
    # load 
    image_np_A = cv2.imread(str(image_pth_A))
    image_np_A = cv2.cvtColor(image_np_A, cv2.COLOR_BGR2RGB)
    image_np_B = cv2.imread(str(image_pth_B))
    image_np_B = cv2.cvtColor(image_np_B, cv2.COLOR_BGR2RGB)

    # expects 3 channels
    if image_np_A.shape[2] == 1:
        image_np_A = np.concatenate([image_np_A] * 3, axis=-1)
    
    # predict
    dets, lms = centerface(image_np_A, threshold=0.35)
    
    det = dets[0]
    width = det[2] - det[0]
    height = det[3] - det[1]
    bigger_dimension= max([width,height])
    width_padding = bigger_dimension - width
    height_padding = bigger_dimension - height
    
    face_A = image_np_A[int(det[1]-width_padding//2):int(det[3]+width_padding//2), \
                 int(det[0]-width_padding//2):int(det[2]+width_padding//2),:]
    face_B = image_np_B[int(det[1]-width_padding//2):int(det[3]+width_padding//2), \
                 int(det[0]-width_padding//2):int(det[2]+width_padding//2),:]
        
    face_A = cv2.resize(face_A, __TARGET_SIZE)
    face_B = cv2.resize(face_B, __TARGET_SIZE).astype(np.uint8)
    face_A_np = np.mean(face_A, axis=2)
    face_A = np.stack([face_A_np,face_A_np,face_A_np], axis=2).astype(np.uint8)
    
    display(Image.fromarray(np.concatenate([face_A, face_B], axis=1)))
        
    return face_A, face_B
    

def operation_Y(image_pth_A, image_pth_B):    
    face_objs_A = DeepFace.extract_faces(
        img_path=image_pth_A,
        target_size=__TARGET_SIZE,
        detector_backend="retinaface",
        enforce_detection=False,
        align=False
    )

    face_B_np = np.mean(cv2.imread(image_pth_B), axis=2)
    face_B_np = np.stack([face_B_np,face_B_np,face_B_np], axis=2)
    
    face_A = (face_objs_A[0]['face'] * 255).astype(np.uint8)
    ar = face_objs_A[0]['facial_area']
    face_B = face_B_np[ar['y']:ar['y']+ar['h'], ar['x']:ar['x']+ar['w']]
    
    face_B = fill_black(face_B)
    face_B = cv2.resize(face_B, __TARGET_SIZE).astype(np.uint8)
    
    face_A_np = np.mean(face_A, axis=2)
    face_A = np.stack([face_A_np,face_A_np,face_A_np], axis=2).astype(np.uint8)
    
    display(Image.fromarray(np.concatenate([face_A, face_B], axis=1)))
        
    return face_A, face_B

# Get the list of patient ids
patient_ids = [name for name in os.listdir(__DATA_FOLDER) if os.path.isdir(os.path.join(__DATA_FOLDER, name))]

# Split the patient ids into train, validation, and test sets
train_ids, test_ids = train_test_split(patient_ids, test_size=0.2, random_state=42)
# train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2



# Function to process and save images
def process_and_save_images(ids, split):
    filepaths_A = []
    filepaths_B = []
    invalid = []
    for _id in sorted(ids):
        patient_folder = os.path.join(__DATA_FOLDER, _id)
        dir_content = [i for i in os.listdir(patient_folder) if i[-3:] == 'bmp']
        image_files = sorted(dir_content, key=lambda x: int(x.split('.')[0]))  # Sort the files to ensure pairs
        image_files = [i for i in image_files if int(i.split('.')[0]) <= 28]
        for file_A, file_B in zip(image_files[0::2], image_files[1::2]):
            print(str(os.path.join(patient_folder, file_A)), str(os.path.join(patient_folder, file_B)))
            
            img_path_A = os.path.join(patient_folder, file_A)
            img_path_B = os.path.join(patient_folder, file_B)
        
            try:
                if random.choice([True, False]):
                    img_A, img_B = operation_X(img_path_A, img_path_B)
                else:
                    img_A, img_B = operation_Y(img_path_A, img_path_B)
            except:
                invalid.append(img_path_A)
                continue
                
            
            # Save the images
            Image.fromarray(img_A).save(os.path.join(split + 'A', f'{_id}_{file_A}'))
            Image.fromarray(img_B).save(os.path.join(split + 'B', f'{_id}_{file_B}'))
            
            filepaths_A.append(str(os.path.join(split + 'A', f'{_id}_{file_A}')))
            filepaths_B.append(str(os.path.join(split + 'B', f'{_id}_{file_B}')))
            
    return filepaths_A, filepaths_B,invalid

2023-12-19 20:23:27.304807: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 20:23:27.357979: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-19 20:23:27.358923: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 20:23:34.272846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Process and save images for each split
out_train = process_and_save_images(train_ids, __TARGET_FOLDER + 'train')
# out_val = process_and_save_images(val_ids, __TARGET_FOLDER + 'val')
out_test = process_and_save_images(test_ids, __TARGET_FOLDER + 'test')